## Procesamiento Datos Pandas

Para esta prueba deberá emplear los datos del archivo candidates.csv para lo cual deberá:

1. Almanecar los datos en una base de datos. (Asegurar el almacenamiento con el tipo de dato apropiado)
2. A partir de la lectura de los datos deberá generar un procesamiento reporte.
    
    (Cree el proceso que consideré más conveniente, realizarlo lo más complejo que le sea posible)
    
3. Realizar un código de envio de correos integrado en su solución.


El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- Yoe (years of experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


In [69]:
import sqlite3
import pandas as pd
import smtplib
from email.mime.text import MIMEText

# Almacenar los datos en una base de datos
def store_data_in_database():
    # archivo CSV con el separador de punto y coma para que no de error
    datos = pd.read_csv('candidates.csv', sep=';', encoding='utf-8')

    # Renombrar las columnas para eliminar los espacios y caracteres especiales
    datos.columns = [column.strip().replace(' ', '_').replace('(', '').replace(')', '') for column in datos.columns]

    conn = sqlite3.connect('candidates.db')

    # Crear una tabla en la base de datos para almacenar los datos
    cursor = conn.cursor()
    create_table_query = '''
        CREATE TABLE IF NOT EXISTS candidates (
            First_Name TEXT,
            Last_Name TEXT,
            Email TEXT,
            Application_Date TEXT,
            Country TEXT,
            YOE TEXT,
            Seniority TEXT,
            Technology TEXT,
            Code_Challenge_Score REAL,
            Technical_Interview_Score REAL
        )
    '''
    cursor.execute(create_table_query)

    # Insertar los datos en la tabla
    datos.to_sql('candidates', conn, if_exists='replace', index=False)

    # Cerrar la conexión a la base de datos
    conn.close()

# Generar un procesamiento de reporte
def generamos_el_reporte():
    # Conectarse a la base de datos SQLite
    conn = sqlite3.connect('candidates.db')

    # Consulta SQL para generar el reporte
    query = '''
        SELECT Country, AVG(Code_Challenge_Score) AS Average_Score
        FROM candidates
        GROUP BY Country
        ORDER BY Average_Score DESC
    '''

    # Ejecutar la consulta y obtener los resultados
    resultados = pd.read_sql_query(query, conn)

    # Cerrar la conexión a la base de datos
    conn.close()

    # Imprimir el reporte
    print("Reporte de promedio de puntuación  de código por país:")
    print(resultados)




In [70]:
# Enviar correo electrónico
def enviar_el_email(sender_email, sender_password, receiver_email, subject, message):
    # Configurar el servidor SMTP de Gmail
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    # Crear el objeto MIMEText con el contenido del mensaje
    msg = MIMEText(message.encode('utf-8'), _charset='utf-8')

    # Configurar los encabezados del correo electrónico
    msg['Subject'] = subject.encode('utf-8')
    msg['From'] = sender_email
    msg['To'] = receiver_email

    # Conectar al servidor SMTP y enviar el correo electrónico
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()





In [ ]:
# Resolución completa del ejercicio
def solve_problem():
    # Almacenar los datos en la base de datos
    store_data_in_database()

    # Generar el procesamiento de reporte
    generamos_el_reporte()

    # Enviar el correo electrónico
    sender_email = 'ejemplo@gmail.com'
    sender_password = 'ejemplocontra'
    receiver_email = 'ejemplo_destino@gmail.com'
    subject = 'Solucion'
    message = 'Adjunto la solucion al problema :D'
    enviar_el_email(sender_email, sender_password, receiver_email, subject, message)

# Llamar a la función principal para resolver el problema
solve_problem()